<a href="https://colab.research.google.com/github/sivasanjeevs/HealthCare-Diagnostics-and-Drug-discovery/blob/main/MLPackage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.138.8.53


In [ ]:
%%writefile app.py
import streamlit as st
import numpy
import pickle
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import pandas

class KMeans:
    def __init__(self,k):
        self.k = k

    def assign_clusters(self,pixels, centroids):
        labels = numpy.zeros(pixels.shape[0], dtype=int)
        for i in range(len(pixels)):
            pixel = pixels[i]
            distances = []
            for centroid in centroids:
                distance = numpy.sqrt((pixel - centroid) ** 2)
                distances.append(distance)
            labels[i] = numpy.argmin(distances)
        return labels

    def update_centroids(self,pixels, labels, k,centroids):
        new_centroids = numpy.zeros((k, 1))
        for i in range(k):
            cluster_points = pixels[labels == i]
            if len(cluster_points) > 0:
                new_centroids[i] = numpy.mean(cluster_points)
            else:
                new_centroids[i] = centroids[i]
        return new_centroids

    def run(self,image):
        k = self.k
        image_resized = cv2.resize(image, (256, 256))
        pixels = image_resized.reshape(-1, 1).astype(float)
        max_iters = 100
        tolerance = 1e-4
        numpy.random.seed(42)
        centroids = pixels[numpy.random.choice(pixels.shape[0], k, replace=False)]

        for i in range(max_iters):
            labels = self.assign_clusters(pixels, centroids)
            new_centroids = self.update_centroids(pixels, labels, k,centroids)
            if numpy.max(numpy.abs(new_centroids - centroids)) < tolerance:
                break
            centroids = new_centroids
        seg_image = labels.reshape(image_resized.shape)

        white_matter = numpy.zeros(seg_image.shape, dtype=numpy.uint8)
        gray_matter = numpy.zeros(seg_image.shape, dtype=numpy.uint8)
        csf_image = numpy.zeros(seg_image.shape, dtype=numpy.uint8)

        white_matter[seg_image == 0] = 255
        gray_matter[seg_image == 1] = 255
        csf_image[seg_image == 2] = 255

        return image_resized,white_matter, gray_matter, csf_image


class DrugEffectiveness:
    def __init__(self, data, condition):
        self.data = data[data['Condition'] == condition].copy()
        self.condition = condition
        self.model_trained = False

    def prepare_data(self):
        self.data['Effective_Binary'] = (self.data['Effective'] > self.data['Effective'].median()).astype(int)
        X = self.data[['EaseOfUse', 'Satisfaction']].values
        y = self.data['Effective_Binary'].values.reshape(-1, 1)
        self.X = numpy.hstack((numpy.ones((X.shape[0], 1)), X))
        self.y = y

    def sigmoid(self, z):
        return 1 / (1 + numpy.exp(-z))

    def matrix_multiply(self, A, B):
        m, n = A.shape
        n_b, p = B.shape
        result = numpy.zeros((m, p))
        for i in range(m):
            for j in range(p):
                result[i, j] = sum(A[i, k] * B[k, j] for k in range(n))
        return result

    def train(self, learning_rate=0.01, max_iters=1000, tolerance=1e-6):
        self.weights = numpy.zeros((self.X.shape[1], 1))
        for iteration in range(max_iters):
            z = self.matrix_multiply(self.X, self.weights)
            predictions = numpy.array([[self.sigmoid(z_i[0])] for z_i in z])
            error = predictions - self.y
            gradient = numpy.zeros(self.weights.shape)
            m = self.X.shape[0]
            for j in range(self.weights.shape[0]):
                gradient[j, 0] = sum(error[i, 0] * self.X[i, j] for i in range(m)) / m
            self.weights -= learning_rate * gradient
            if sum(abs(g) for g in gradient.flatten()) < tolerance:
                print(f"Convergence reached at iteration {iteration}")
                break
        self.model_trained = True

    def predict(self, X):
        z = self.matrix_multiply(X, self.weights)
        return numpy.array([[1 if self.sigmoid(z_i[0]) >= 0.5 else 0] for z_i in z])

    def recommend_drug(self):
        if not self.model_trained:
            print("Model not trained. Training now...")
            self.train()
        self.data['Prediction'] = self.predict(self.X)
        effectiveness = self.data.groupby('Drug')['Prediction'].mean().sort_values(ascending=False)
        recommended_drug = effectiveness.idxmax()
        return recommended_drug,effectiveness


st.set_page_config(layout="wide")
def home():
    st.title('Health Diagnostics and Drug Discovery')
    st.write("Choose one of the options")

    if st.button('Disease Prediction'):
        st.session_state.page = 'page_1'

    if st.button('Drug Effectiveness'):
        st.session_state.page = 'page_2'

    if st.button('Brain MRI Segmentation'):
        st.session_state.page = 'page_3'


def page_1():
    st.title("Disease Prediction")
    valid_strings = ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs', 'fast_heart_rate', 'pain_during_bowel_movements', 'pain_in_anal_region', 'bloody_stool', 'irritation_in_anus', 'neck_pain', 'dizziness', 'cramps', 'bruising', 'obesity', 'swollen_legs', 'swollen_blood_vessels', 'puffy_face_and_eyes', 'enlarged_thyroid', 'brittle_nails', 'swollen_extremeties', 'excessive_hunger', 'extra_marital_contacts', 'drying_and_tingling_lips', 'slurred_speech', 'knee_pain', 'hip_joint_pain', 'muscle_weakness', 'stiff_neck', 'swelling_joints', 'movement_stiffness', 'spinning_movements', 'loss_of_balance', 'unsteadiness', 'weakness_of_one_body_side', 'loss_of_smell', 'bladder_discomfort', 'foul_smell_of urine', 'continuous_feel_of_urine', 'passage_of_gases', 'internal_itching', 'toxic_look_(typhos)', 'depression', 'irritability', 'muscle_pain', 'altered_sensorium', 'red_spots_over_body', 'belly_pain', 'abnormal_menstruation', 'dischromic _patches', 'watering_from_eyes', 'increased_appetite', 'polyuria', 'family_history', 'mucoid_sputum', 'rusty_sputum', 'lack_of_concentration', 'visual_disturbances', 'receiving_blood_transfusion', 'receiving_unsterile_injections', 'coma', 'stomach_bleeding', 'distention_of_abdomen', 'history_of_alcohol_consumption', 'fluid_overload', 'blood_in_sputum', 'prominent_veins_on_calf', 'palpitations', 'painful_walking', 'pus_filled_pimples', 'blackheads', 'scurring', 'skin_peeling', 'silver_like_dusting', 'small_dents_in_nails', 'inflammatory_nails', 'blister', 'red_sore_around_nose', 'yellow_crust_ooze']
    user_input = st.multiselect("Enter the symptoms",valid_strings)

    if st.button('Submit'):
      st.write(user_input)

      prediction=numpy.zeros(len(valid_strings)-1)
      for i in user_input:
        index=valid_strings.index(i)
        prediction[index]=1

      with open('/content/drive/MyDrive/ML/Package/model.pkl', 'rb') as f:
          model=pickle.load(f)

      st.write('Predicted Disease:',model.predict(prediction.reshape(1,-1)))

    if st.button('Home'):
        st.session_state.page = 'home'

def page_2():
    st.title("Drug Effectiveness")

    valid_strings=['Acute Bacterial Sinusitis', 'adenocarcinoma of pancreas', 'Atopic Dermatitis', 'back pain', 'Bacterial Conjunctivitis', 'Bacterial Urinary Tract Infection', 'biliary calculus', 'chickenpox', 'colorectal cancer', 'depression', 'diverticulitis of gastrointestinal tract', 'edema', 'endometriosis', 'fever', 'fibromyalgia', 'flatulence', 'furunculosis', 'gastroesophageal reflux disease', 'genital herpes simplex', 'gout', 'hemorrhoids', 'herpes zoster', 'hypercholesterolemia', 'hypertension', 'impetigo', 'Infantile Autism', 'Influenza', 'meniere\'s disease', 'oral candidiasis', 'Pharyngitis due to Streptococcus Pyogenes', 'prevention of cerebrovascular accident', 'pyelonephritis', 'scabies', 'Sleepiness Due To Obstructive Sleep Apnea', 'sore throat', 'vertigo', 'vulvovaginal candidiasis']
    user_input = st.selectbox("Enter the condition",valid_strings)
    if st.button('Submit'):
      st.write(user_input)

      data = pandas.read_csv('/content/drive/MyDrive/ML/Package/Drug.csv')
      model = DrugEffectiveness(data, user_input)
      model.prepare_data()
      recommended_drug,effectiveness=model.recommend_drug()

      st.write('Recommended Drug:',recommended_drug)
      effectiveness.plot(kind='bar', title=f'Effectiveness of Drugs for {user_input}')
      plt.xlabel('Drug')
      plt.ylabel('Predicted Effectiveness')
      st.pyplot(plt)

    if st.button('Home'):
        st.session_state.page = 'home'

def page_3():
    st.title('Brain MRI Segmentation')

    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:

        image = cv2.imdecode(numpy.frombuffer(uploaded_file.read(),numpy.uint8), cv2.IMREAD_GRAYSCALE)

        kmeans = KMeans(k=3)
        original_image, white_matter, gray_matter, csf_image = kmeans.run(image)

        fig, axs = plt.subplots(1, 4, figsize=(20, 5))

        axs[0].imshow(original_image, cmap='gray')
        axs[0].set_title("Original Image")
        axs[0].axis('off')

        axs[1].imshow(white_matter, cmap='gray')
        axs[1].set_title("White Matter")
        axs[1].axis('off')

        axs[2].imshow(gray_matter, cmap='gray')
        axs[2].set_title("Gray Matter")
        axs[2].axis('off')

        axs[3].imshow(csf_image, cmap='gray')
        axs[3].set_title("Cerebrospinal Fluid (CSF)")
        axs[3].axis('off')

        st.pyplot(fig)

    if st.button('Home'):
        st.session_state.page = 'home'


if 'page' not in st.session_state:
    st.session_state.page = 'home'

if st.session_state.page == 'home':
    home()
elif st.session_state.page == 'page_1':
    page_1()
elif st.session_state.page == 'page_2':
    page_2()
elif st.session_state.page == 'page_3':
    page_3()

Overwriting app.py


In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 579ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.138.8.53
your url is: https://bitter-carrots-vanish.loca.lt
